In [89]:
import itertools
import numpy as np
import pandas as pd
import os, sys
from scipy.stats import pearsonr, spearmanr
import pickle
from collections import defaultdict
from benchmark_utils import tile_regions, parse_bed_file_with_coords
import subprocess

np.random.seed(0)

In [2]:
test_tissues = [46, 47, 49, 50, 54, 105, 159, 160, 161, 174, 202, 203, 211, 212, 213,
                 214, 239, 267, 268, 275, 276, 277, 278, 288, 318, 319, 320, 321,
                 323, 324, 422, 442, 443, 473, 474, 515, 517]

valid_tissues = [17, 39, 60, 66, 70, 71, 72, 73, 82, 97, 98, 99, 136, 137, 227,
                       236, 240, 306, 307, 312, 339, 340, 341, 342, 349, 425, 458, 482]

easytest_tissues = [3, 25, 64, 87, 90, 98, 106, 115, 124, 137, 159, 189, 192, 247,
                     283, 284, 300, 311, 332, 334, 345, 448, 467, 480, 481, 532, 582]

excluded_tissues = test_tissues + valid_tissues + easytest_tissues

assays = ['dnase', 'atac', 'h3k4me1', 'h3k4me2', 'h3k4me3', 'h3k9ac', 'h3k9me3', 'h3k27ac', 'h3k27me3', 'h3k36me3', 'h3k79me2',
          'ctcf', 'cage', 'rampage', 'rna_total', 'rna_polya', 'rna_10x', 'wgbs']
celltypes = list(range(0,392))

In [150]:
subset_bed_path='/project/deeprna_data/avocado_data_trainingfolds/selected_regions.bed'

subset_bed = pd.read_csv(
    subset_bed_path,
    sep="\t",
    header=None,
    names=["chr", "start", "end", "fold_id"]
)

In [14]:
with open('/project/deeprna/benchmark/avocado_models/training_baselines.pk', 'rb') as f:
    training_baseline = pickle.load(f)

In [15]:
with open('/project/deeprna/benchmark/avocado_predictions/avocado_testtissues_trainingfolds_epoch_2900_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f, encoding='latin1')

In [27]:
# Make sure that the data is correct, it should be 8192 bins for each sequence, all concatanated.
print(training_baseline['dnase'].shape[0] / len(sel_idx))
print(predictions['dnase'][323].shape[0] / len(sel_idx))

8192.0
8192.0


In [83]:
training_baseline['dnase'].shape[0]

5611520

In [151]:
# Load BED regions.
bed_regions = parse_bed_file_with_coords(subset_bed_path)
print(f"Parsed {len(bed_regions)} regions from BED file.")

# Tile regions for predictions.
pred_tiles = tile_regions(bed_regions, 64, 64, drop_last=True)
pred_tiles = [(a,str(b), str(c)) for (a,b,c) in pred_tiles]
print(f"Generated {len(pred_tiles)} prediction tiles.")

Parsed 685 regions from BED file.
Generated 5611520 prediction tiles.


In [152]:
with open('/project/deeprna/benchmark/avocado_tiles.bed', 'w') as f:
    f.write('\n'.join(['\t'.join(x) for x in pred_tiles]))

In [ ]:
results = {'tissue':[], 'assay':[], 'model':[], 'pearson':[], 'spearman':[]}

for assay in predictions:
    for tissue in predictions[assay]:
        true = np.load(f'/project/deeprna_data/avocado_data_trainingfolds/tissue_{tissue}_{assay}.npy')
        corgi = pd.read_csv(f'/project/deeprna_data/avocado_vs_corgi/tissue{tissue}_{assay}_corgi.bed', sep='\t')
        avail = corgi.loc[corgi['mean'].isna() == False].index

        y = {}
        y['corgi'] = corgi.loc[avail, 'mean'].values.astype(float)
        y['avocado'] = predictions[assay][tissue][avail].astype(float)
        y['baseline'] = training_baseline[assay][avail].astype(float)
        y['true'] = true[avail].astype(float)

        for model in ['corgi', 'avocado', 'baseline']:
            p = round(pearsonr(y['true'], y[model])[0], 3)
            s = round(spearmanr(y['true'], y[model])[0], 3)
            
            results['tissue'].append(tissue)
            results['assay'].append(assay)
            results['model'].append(model)
            results['pearson'].append(p)
            results['spearman'].append(s)

In [210]:
results_df = pd.DataFrame.from_dict(results)

In [211]:
results_df.to_csv('/project/deeprna/benchmark/avocado_vs_corgi_results.csv')

In [213]:
results_df.drop('assay', axis=1).groupby(by='model').mean()

,tissue,pearson,spearman
model,,,
avocado,203.834146,0.623215,0.360546
baseline,203.834146,0.635961,0.407712
corgi,203.834146,0.624620,0.423556


In [218]:
results_df.drop('assay', axis=1).groupby(by='model').mean().loc['avocado', 'spearman']

0.3605463414634147

In [212]:
results_df.groupby(by=['assay', 'model']).mean()

tissue   pearson  spearman
assay    model                                   
atac     avocado   241.888889  0.648667  0.631556
         baseline  241.888889  0.671667  0.680556
         corgi     241.888889  0.653556  0.642889
cage     avocado   231.000000  0.505571  0.085286
         baseline  231.000000  0.585143  0.188857
         corgi     231.000000  0.556714  0.306714
ctcf     avocado   228.521739  0.676087  0.262130
         baseline  228.521739  0.676174  0.261783
         corgi     228.521739  0.649913  0.284565
dnase    avocado   206.052632  0.849316  0.655421
         baseline  206.052632  0.868737  0.750211
         corgi     206.052632  0.841368  0.727000
h3k27ac  avocado   201.700000  0.611400  0.341050
         baseline  201.700000  0.594950  0.379850
         corgi     201.700000  0.595550  0.386250
h3k27me3 avocado   177.176471  0.435882  0.320235
         baseline  177.176471  0.440941  0.284941
         corgi     177.176471  0.459059  0.331176
h3k36me3 avocado   184.750000  0.562875  0.379625
         baseline  184.750000  0.556562  0.375937
         corgi     184.750000  0.546188  0.372375
h3k4me1  avocado   194.833333  0.432056  0.369556
         baseline  194.833333  0.421333  0.365778
         corgi     194.833333  0.445056  0.396556
h3k4me2  avocado   116.800000  0.774600  0.404200
         baseline  116.800000  0.734800  0.359600
         corgi     116.800000  0.774000  0.387000
h3k4me3  avocado   209.277778  0.850000  0.240611
         baseline  209.277778  0.849556  0.293667
         corgi     209.277778  0.831167  0.310778
h3k79me2 avocado   131.166667  0.725667  0.476500
         baseline  131.166667  0.707333  0.455667
         corgi     131.166667  0.687500  0.454667
h3k9ac   avocado   135.500000  0.697000  0.462500
         baseline  135.500000  0.695000  0.433000
         corgi     135.500000  0.673750  0.417500
h3k9me3  avocado   208.357143  0.412286  0.342714
         baseline  208.357143  0.390000  0.333357
         corgi     208.357143  0.387000  0.343929
rampage  avocado   235.578947  0.516105  0.009000
         baseline  235.578947  0.622474  0.253737
         corgi     235.578947  0.614947  0.382789
wgbs     avocado   223.900000  0.838100  0.812800
         baseline  223.900000  0.901200  0.978100
         corgi     223.900000  0.811600  0.807200

In [216]:
a = results_df.drop('assay', axis=1).groupby(by=['tissue', 'model']).mean()

In [217]:
a.to_csv('/project/deeprna/benchmark/avocado_corgi_by_tissue.csv', sep='\t')

In [242]:
# Tissue specific regions

els = pd.read_csv('/project/deeprna/benchmark/avocado_tiles_encodeels.bed', sep='\t', names=['chrom', 'start', 'end'])
els.head()

,chrom,start,end
0,chr12,740436,740500
1,chr12,740500,740564
2,chr12,740564,740628
3,chr12,740628,740692
4,chr12,740692,740756


In [235]:
corgi.columns = ['chrom', 'start', 'end', 'size', 'num_data', 'min', 'max', 'mean', 'median', 'sum']
corgi['index'] = corgi.index

In [241]:
els_df = corgi.merge(els, on=['chrom', 'start', 'end'], how='inner')
els_idx = els_df.loc[els_df['mean'].isna() == False, 'index'].values

In [243]:
results_els = {'tissue':[], 'assay':[], 'model':[], 'pearson':[], 'spearman':[]}

for assay in predictions:
    for tissue in predictions[assay]:
        true = numpy.load(f'/project/deeprna_data/avocado_data_trainingfolds/tissue_{tissue}_{assay}.npy')
        corgi = pd.read_csv(f'/project/deeprna_data/avocado_vs_corgi/tissue{tissue}_{assay}_corgi.bed', sep='\t')
        avail = els_idx

        y = {}
        y['corgi'] = corgi.loc[avail, 'mean'].values.astype(float)
        y['avocado'] = predictions[assay][tissue][avail].astype(float)
        y['baseline'] = training_baseline[assay][avail].astype(float)
        y['true'] = true[avail].astype(float)

        for model in ['corgi', 'avocado', 'baseline']:
            p = round(pearsonr(y['true'], y[model])[0], 3)
            s = round(spearmanr(y['true'], y[model])[0], 3)
            
            results_els['tissue'].append(tissue)
            results_els['assay'].append(assay)
            results_els['model'].append(model)
            results_els['pearson'].append(p)
            results_els['spearman'].append(s)

In [244]:
results_els_df = pd.DataFrame.from_dict(results_els)

In [245]:
results_els_df.drop('assay', axis=1).groupby(by='model').mean()

,tissue,pearson,spearman
model,,,
avocado,203.834146,0.588693,0.400112
baseline,203.834146,0.614956,0.434371
corgi,203.834146,0.613434,0.458610


In [246]:
results_els_df.groupby(by=['assay', 'model']).mean()

tissue   pearson  spearman
assay    model                                   
atac     avocado   241.888889  0.597111  0.592333
         baseline  241.888889  0.608778  0.622556
         corgi     241.888889  0.601333  0.595667
cage     avocado   231.000000  0.182571  0.147429
         baseline  231.000000  0.399143  0.253000
         corgi     231.000000  0.389714  0.338000
ctcf     avocado   228.521739  0.760478  0.302609
         baseline  228.521739  0.762261  0.322000
         corgi     228.521739  0.732043  0.319609
dnase    avocado   206.052632  0.792316  0.695000
         baseline  206.052632  0.804895  0.695579
         corgi     206.052632  0.775842  0.718526
h3k27ac  avocado   201.700000  0.657500  0.463300
         baseline  201.700000  0.620300  0.428950
         corgi     201.700000  0.628350  0.467350
h3k27me3 avocado   177.176471  0.547353  0.334235
         baseline  177.176471  0.576588  0.338529
         corgi     177.176471  0.576824  0.368824
h3k36me3 avocado   184.750000  0.581500  0.380063
         baseline  184.750000  0.578812  0.390375
         corgi     184.750000  0.570375  0.400625
h3k4me1  avocado   194.833333  0.444722  0.430833
         baseline  194.833333  0.422000  0.399278
         corgi     194.833333  0.450833  0.443222
h3k4me2  avocado   116.800000  0.794400  0.503400
         baseline  116.800000  0.748400  0.421600
         corgi     116.800000  0.797200  0.518400
h3k4me3  avocado   209.277778  0.890333  0.379500
         baseline  209.277778  0.888556  0.406444
         corgi     209.277778  0.869444  0.426944
h3k79me2 avocado   131.166667  0.753667  0.512500
         baseline  131.166667  0.744833  0.515500
         corgi     131.166667  0.722000  0.528667
h3k9ac   avocado   135.500000  0.759750  0.441250
         baseline  135.500000  0.749500  0.432250
         corgi     135.500000  0.727250  0.423750
h3k9me3  avocado   208.357143  0.316143  0.248143
         baseline  208.357143  0.302643  0.249071
         corgi     208.357143  0.327429  0.308357
rampage  avocado   235.578947  0.089263  0.031526
         baseline  235.578947  0.299842  0.313842
         corgi     235.578947  0.361684  0.397526
wgbs     avocado   223.900000  0.804000  0.845000
         baseline  223.900000  0.876600  0.958700
         corgi     223.900000  0.808900  0.826600